In [10]:
!pip install prettytable


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import os
import torch as th
from stable_baselines3 import PPO, SAC, TD3
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv, VecNormalize
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback
from envs.bidding import BiddingEnv  # Assuming this is your custom environment
from tabulate import tabulate
import numpy as np
# from prettytable import PrettyTable

## Space Debugging

In [2]:
log_dir = "./runs/baselines"


env = BiddingEnv(render_mode='human')

# Wrap with VecNormalize for observation and reward normalization
vec_env = DummyVecEnv([lambda: env])  # Single process
vec_env = VecNormalize(vec_env, norm_obs=True, norm_reward=True)


# Load the pretrained model
model = SAC.load("./runs/baselines/SAC/best_model", env=vec_env)

In [3]:
from stable_baselines3.common.env_checker import check_env
from envs.bidding import BiddingEnv
env = BiddingEnv()
obs = env.reset()

print(obs[1])

env.render(mode='human')

action, _ = model.predict(obs[0], deterministic=True) # use pretrained model
# action = env.action_space.sample() # sample random action

obs, reward, done, truncated, info = env.step(action)
env.render(mode='human')
print("Completed, final reward =", reward)
print('Optimal reward = ', env.optimal_reward())
obs, reward, done, truncated, info = env.step(env.action_space.sample())
print("Random Reward: ", reward)


{}
╒══════════╤══════════╤══════════╤══════════╤══════════╤══════════╕
│   Task 1 │   Task 2 │   Task 3 │   Task 4 │   Task 5 │   Task 6 │
╞══════════╪══════════╪══════════╪══════════╪══════════╪══════════╡
│        0 │        0 │        0 │        0 │        0 │        0 │
├──────────┼──────────┼──────────┼──────────┼──────────┼──────────┤
│        0 │        0 │        0 │        0 │        0 │        0 │
├──────────┼──────────┼──────────┼──────────┼──────────┼──────────┤
│        0 │        0 │        0 │        0 │        0 │        0 │
├──────────┼──────────┼──────────┼──────────┼──────────┼──────────┤
│        0 │        0 │        0 │        0 │        0 │        0 │
├──────────┼──────────┼──────────┼──────────┼──────────┼──────────┤
│        0 │        0 │        0 │        0 │        0 │        0 │
├──────────┼──────────┼──────────┼──────────┼──────────┼──────────┤
│        0 │        0 │        0 │        0 │        0 │        0 │
├──────────┼──────────┼──────────┼──────────┼

AssertionError: Bidding space is invalid: [[0.00000000e+00 1.00000000e+01 1.00000000e+01 1.18066370e-01
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.00000000e+01 0.00000000e+00 8.50856304e-04
  1.00000000e+01 0.00000000e+00]
 [1.00000000e+01 1.00000000e+01 1.00000000e+01 1.00000000e+01
  0.00000000e+00 1.00000000e+01]
 [1.00000000e+01 0.00000000e+00 1.00000000e+01 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.00000000e+01 0.00000000e+00 1.00000000e+01 0.00000000e+00
  0.00000000e+00 1.00000000e+01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+01
  1.00000000e+01 1.00000000e+01]
 [1.00000000e+01 1.00000000e+01 0.00000000e+00 1.00000000e+01
  1.00000000e+01 1.00000000e+01]
 [0.00000000e+00 1.00000000e+01 0.00000000e+00 1.00000000e+01
  1.00000000e+01 3.57627869e-06]
 [1.00000000e+01 0.00000000e+00 1.00000000e+01 0.00000000e+00
  1.00000000e+01 0.00000000e+00]]

## Training Debugging

In [ ]:
# Create the environment
env = BiddingEnv()

obs, _ = env.reset()

assert env.observation_space.contains(obs), f'obs not right {obs}'
assert env.action_space.contains(env.bidding_matrix), f'action not right {env.bidding_matrix}'

print(obs)

# SETUP THE MODEL

log_dir = "./runs/baselines"
os.makedirs(log_dir, exist_ok=True)
env = Monitor(env, log_dir)

vec_env = DummyVecEnv([lambda: env])

model = SAC(
        "MultiInputPolicy",  # Use MultiInputPolicy for Dict observation spaces
        vec_env,
        verbose=1,
        tensorboard_log=log_dir,
        device="auto",
    )

model.learn(
    total_timesteps=int(10),  # 1M steps
)

model.save(os.path.join(log_dir, f"SAC_debug_model"))

# EVALUATE THE MODEL

eval_env = DummyVecEnv([lambda: env])
mean_reward, std_reward = model.evaluate_policy(model, eval_env, n_eval_episodes=10)
print(f"Mean reward: {mean_reward}, Std reward: {std_reward}")

for i in range(100):
    action, _ = model.predict(obs, deterministic=True)

    if env.action_space.contains(action):
        pass
    else:
        print(f"Invalid action on step {i}:", action)
        raise ValueError("Invalid action")

print('completed')

{'robot_positions': array([[5, 8],
       [5, 0],
       [1, 7],
       [6, 9],
       [4, 5],
       [4, 2],
       [7, 7],
       [1, 7],
       [0, 6]], dtype=int32), 'robot_types': array([1, 0, 1, 2, 2, 0, 1, 2, 1], dtype=int32), 'task_positions': array([[9, 9],
       [6, 9],
       [1, 8],
       [3, 9],
       [7, 3],
       [1, 9]], dtype=int32), 'task_prizes': array([ 8,  2, 89, 73, 71,  4], dtype=int32), 'task_types': array([1, 0, 1, 2, 1, 0], dtype=int32)}
Using cpu device
Logging to ./runs/baselines/SAC_2


AssertionError: Invalid action: [[ 6.  0.  9.  5. 10.  7.]
 [ 4. 10.  9.  5.  5.  0.]
 [ 9.  6.  6.  6.  6.  1.]
 [ 8.  9.  0.  2.  3.  0.]
 [ 2.  9. 10.  8.  5.  8.]
 [ 8.  2.  1.  6.  9. 10.]
 [ 3.  2.  9.  1.  7.  0.]
 [ 8.  4.  2.  2.  0.  2.]
 [10.  9.  0.  8.  4.  9.]]